In [1]:
import numpy as np

In [2]:
class LDUDecomposition:
    def __init__(self, matrix):
        self.A = np.array(matrix, dtype=float)
    """Single-responsibility LDU factorization with unit L and Uhat."""
    def LDU(self,A):
        """
        Input:
            A : (n, n) array_like
        Output:
            L, D, Uhat  such that  A = L @ D @ Uhat
            L    : unit lower-triangular
            D    : diagonal
            Uhat : unit upper-triangular
        """
        
    # Work on a float copy of A
        U = np.array(self.A, dtype=float, copy=True)
        n = U.shape[0]

        if np.isclose(np.linalg.det(A), 0.0):
            raise ValueError("LDU decomposition does not exist for singular matrices (det(A) = 0).")
        
    # Create L as identity
        L = np.eye(n, dtype=float)

    # ----- Forward elimination -----
        for i in range(n):
            pivot = U[i, i]
            if pivot == 0.0:
                raise ZeroDivisionError(f"Zero pivot encountered at i={i}. Cannot proceed.")
            for j in range(i + 1, n):
            # Store multiplier in L
                L[j, i] = U[j, i] / pivot
            # Eliminate below pivot
                U[j, :] -= L[j, i] * U[i, :]

    # Extract D (diagonal part) and make Uhat unit upper
        D = np.eye(n, dtype=float)
        Uhat = U.copy()
        for k in range(n):
            D[k, k] = U[k, k]
            if D[k, k] == 0.0:
                raise ZeroDivisionError(f"Zero diagonal in U at k={k}; cannot form Uhat.")
            Uhat[k, :] /= D[k, k]

        return L, D, Uhat

In [4]:
class LeastSquaresSVD:
    #(Dependency Inversion Principle: depends on numpy only).
    def solve(self, A, b):
        """
        U, Sigma, V = SVD(A)
        rk(A) = # nonzero elements of Sigma
        C = U^T b
        y = zeros(n)
        for i = 1..rk(A):
            y[i] = C[i] / Sigma[i,i]
        x = V y
        return x
        """
        
        A = np.asarray(A, dtype=float)
        b = np.asarray(b, dtype=float).reshape(-1)
        m, n = A.shape

        # U, Sigma, V = SVD(A)
        obj=ManualSVD(A)
        U, Sigma, V = obj.compute_svd()
        
        # rk(A) = # nonzero elements of Sigma (exact, no tolerance)
        s = np.diag(Sigma)                  # singular values
        r = int(np.sum(s != 0.0))

        # C = U^T b
        C = U.T @ b

        # y = zeros(n); y[i] = C[i]/Sigma[i,i] for i = 0..r-1
        y = np.zeros(n, dtype=float)
        for i in range(r):
            y[i] = C[i] / Sigma[i, i]

        # x = V y
        x = V @ y
        return x
